# Develop an Alpha Zero Game Player - Servers
### Projeto 2 - Laboratórios de IA e CD

### Ana Couto, André Vieira, Iara Silva, Mariana Serrão

## Introdução


sdfghj

## Jogo Ataxx

Interface gráfica para o jogo Ataxx.

In [1]:
from tkinter import *
import numpy as np
import copy
import random as r

b_w="BLUE WINS!!"
r_w="RED WINS!!"

print("Escolha número de linhas/colunas (4 ou 6):")
NB = int(input())  # Board number of rows/columns
size_of_board = 600
size_of_square = size_of_board/NB
symbol_size = (size_of_square*0.75-10)/2
symbol_thickness = 20
blue_color = '#496BAB'
red_color = '#F33E30'

possible_moves_global=[]
position_global=[]
bool=False
origin_pos=[]
moves_blue_global=[]
moves_red_global=[]
blue_pieces=[]
red_pieces=[]
board2=[]

class ataxx():
    def __init__(self):
        self.window = Tk()
        self.window.title('Ataxx')
        self.canvas = Canvas(self.window, width=size_of_board, height=size_of_board, background="white")
        self.canvas.pack()
        self.window.bind('<Button-1>', self.click)
        self.board = np.zeros(shape=(NB, NB))
        self.board[0][0]=2
        self.board[0][NB-1]=1
        self.board[NB-1][NB-1]=1
        self.board[NB-1][0]=2
        self.player_blue_turn = True
        self.game_ended = False
        self.init_draw_board()
        self.mainloop()


    def mainloop(self):
        while not self.game_ended:
            self.window.update()

    #----------------DESENHO DO TABULEIRO---------------------------------------------------------------------------------------------------------

    def init_draw_board(self):
        self.canvas.delete("all")
        for i in range(NB-1):
            self.canvas.create_line((i+1)*size_of_square, 0, (i+1)*size_of_square, size_of_board)
        for i in range(NB-1):
            self.canvas.create_line(0,(i+1)*size_of_square, size_of_board, (i+1)*size_of_square)
        self.canvas.create_oval(size_of_square/2 - symbol_size, size_of_square/2 - symbol_size,
                                size_of_square/2 + symbol_size, size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=red_color,
                                fill=red_color)
        self.canvas.create_oval(size_of_board - size_of_square/2 - symbol_size,size_of_board - size_of_square/2 - symbol_size,
                                size_of_board - size_of_square/2 + symbol_size, size_of_board - size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=blue_color,
                                fill=blue_color)
        self.canvas.create_oval(size_of_square/2 - symbol_size,size_of_board - size_of_square/2 - symbol_size,
                                size_of_square/2 + symbol_size, size_of_board - size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=blue_color,
                                fill=blue_color)
        self.canvas.create_oval(size_of_board - size_of_square/2 - symbol_size, size_of_square/2- symbol_size,
                                size_of_board - size_of_square/2 + symbol_size, size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=red_color,
                                fill=red_color)


    def update_board(self, x, y, origin):
        for i in range(max(0, x-1), min(NB, x+2)):
            for j in range(max(0, y-1), min(NB, y+2)):
                if not self.is_square_clear([i,j]):
                    if self.player_blue_turn:
                        self.draw_blue([i,j])
                    else:
                        self.draw_red([i,j])
                    self.board[i][j]=self.board[x][y]
        if x-origin[0]== 2 or y-origin[1]== 2 or x-origin[0]== -2 or y-origin[1]== -2:
            self.board[origin[0]][origin[1]]=0
            pos=self.convert_logical_to_grid_position(origin)
            self.draw_whitespace(pos)
        self.score()
        self.all_moves()
        self.player_blue_turn = not self.player_blue_turn

    def all_moves(self):
        global moves_blue_global
        global moves_red_global
        global blue_pieces
        global red_pieces
        for i in range(NB):
            for j in range(NB):
                if self.board[i][j]==1:
                    moves_blue_global.append(self.possible_moves([i,j]))
                    blue_pieces.append([i,j])
                elif self.board[i][j]==2:
                    moves_red_global.append(self.possible_moves([i,j]))
                    red_pieces.append([i,j])
        if len(moves_blue_global)==0:
            self.no_moves(1)
        elif len(moves_red_global)==0:
            self.no_moves(2)
        moves_blue_global=[]
        moves_red_global=[]

    def no_moves(self, player):
        if player==1:
            for i in range(NB):
                for j in range(NB):
                    if self.board[i][j]==0:
                        self.board[i][j]=2
                        self.draw_red([i,j])
        elif player==2:
            for i in range(NB):
                for j in range(NB):
                    if self.board[i][j]==0:
                        self.board[i][j]=1
                        self.draw_blue([i,j])
        self.score()

    def execute_move(self, move, origin, player):
        self.board[move[0]][move[1]] = player
        self.update_board(move[0], move[1], origin)

    def is_square_clear(self, pos):
        if not np.array_equal(pos, []):
            return self.board[pos[0]][pos[1]] == 0

    def valid_move(self, logical_pos):
        return self.is_square_clear(logical_pos)

    #dado a peça selecionada, devolve uma lista
    #com todos os movimentos possiveis da mesma
    def possible_moves(self, move):
        possible_moves=[]
        for i in range(max(0,move[0]-2), min(NB, move[0]+3)):
            for j in range(max(0,move[1]-2), min(NB, move[1]+3)):

                if self.is_square_clear([i,j]):
                    possible_moves.append([i,j])

        return possible_moves

    def score(self):
        cont_blue=0
        cont_red=0
        cheio=True
        for i in range(NB):
            for j in range(NB):
                if self.board[i][j]==1:
                    cont_blue+=1
                elif self.board[i][j]==2:
                    cont_red+=1
                if self.board[i][j]==0:
                    cheio=False
        print("Blue score= ",  cont_blue)
        print("Red score= ",  cont_red)
        print("------------------------")
        self.window.title("Ataxx - Red : %d vs %d : Blue" % (cont_red, cont_blue))
        if cont_blue==0:
            self.game_is_over(cont_red, cont_blue)
        elif cont_red==0:
            self.game_is_over(cont_red, cont_blue)
        elif cheio:
            self.game_is_over(cont_red, cont_blue)


    def game_is_over(self, red, blue):
        self.game_ended = True
        print()
        if blue>red:
            print(b_w)
        else:
            print(r_w)
        self.clear_possible_moves()

#----------------------TRANSFORMAR EM MATRIZ PARA APLICAR REGRAS---------------

    def convert_logical_to_grid_position(self, logical_pos):
        logical_pos = np.array(logical_pos, dtype=int)
        return np.array((size_of_square)*logical_pos + size_of_square/2)

    def convert_grid_to_logical_position(self, grid_pos):
        grid_pos = np.array(grid_pos)
        return np.array(grid_pos//size_of_square, dtype=int)

#-----------------------DESENHAR PECAS----------------------------------------
    def draw_whitespace(self, grid_pos):



        self.canvas.create_rectangle(grid_pos[0] - symbol_size, grid_pos[1] - symbol_size,
                            grid_pos[0] + symbol_size, grid_pos[1] + symbol_size,
                            width=symbol_thickness, outline="white",
                            fill="white")


    def draw_blue(self, logical_pos):
        logical_pos = np.array(logical_pos)
        grid_pos = self.convert_logical_to_grid_position(logical_pos)
        self.canvas.create_oval(grid_pos[0] - symbol_size, grid_pos[1] - symbol_size,
                            grid_pos[0] + symbol_size, grid_pos[1] + symbol_size,
                            width=symbol_thickness, outline=blue_color,
                            fill=blue_color)

    def draw_red(self, logical_pos):
        logical_pos = np.array(logical_pos)
        grid_pos = self.convert_logical_to_grid_position(logical_pos)
        self.canvas.create_oval(grid_pos[0] - symbol_size, grid_pos[1] - symbol_size,
                            grid_pos[0] + symbol_size, grid_pos[1] + symbol_size,
                            width=symbol_thickness, outline=red_color,
                            fill=red_color)


#----------------------------------- Verificaçao movimentos e jogadas ------------------------------------

    def draw_possible_moves(self, possible_moves):

        # desenha no tabuleiro as jogadas possiveis da bola selecionada

        moves=[0]*len(possible_moves)
        for i in range(len(possible_moves)):
            moves[i]=self.convert_logical_to_grid_position(possible_moves[i])
            self.canvas.create_oval(moves[i][0]-symbol_size, moves[i][1] - symbol_size,
                                    moves[i][0]+symbol_size, moves[i][1]+ symbol_size,
                                    width=symbol_thickness, outline="gray", fill="gray", tags="possible")


    def clear_possible_moves(self):
        self.canvas.delete("possible")

#----------------------- MOUSE -----------------------------------------------------------

    def click(self, event):
        if self.game_ended: return
        grid_pos = [event.x, event.y]
        logical_pos = self.convert_grid_to_logical_position(grid_pos)
        global origin_pos
        global possible_moves_global
        origin_pos = logical_pos

        if self.board[logical_pos[0]][logical_pos[1]] == 1 and self.player_blue_turn:
            possible_moves_global = self.possible_moves(logical_pos)

            if not np.array_equal(possible_moves_global, []):
                self.window.bind("<Button-1>", self.second_click)
            self.draw_possible_moves(possible_moves_global)

        elif self.board[logical_pos[0]][logical_pos[1]] == 2 and not self.player_blue_turn:
            possible_moves_global = self.possible_moves(logical_pos)

            if not np.array_equal(possible_moves_global, []):
                self.window.bind("<Button-1>", self.second_click)
            self.draw_possible_moves(possible_moves_global)


    def second_click(self, event):
        if self.game_ended: return
        global bool
        grid_pos = [event.x, event.y]
        logical_pos = self.convert_grid_to_logical_position(grid_pos)
        global possible_moves_global
        possible_moves_global=np.array(possible_moves_global, dtype=int)

        for element in possible_moves_global:

            if np.array_equal(logical_pos, element):
                global position_global
                position_global = logical_pos
                bool=True
        bool = True
        self.click2()
        possible_moves_global=[]
        position_global=[]


    def click2(self):
        if self.game_ended: return
        global bool

        if self.player_blue_turn:
            player=1
        else:
            player=2

        if self.valid_move(position_global):

            if self.second_click_pressed(bool):
                if self.player_blue_turn and self.board[origin_pos[0]][origin_pos[1]] == 1:
                    self.draw_blue(position_global)
                    self.execute_move(position_global, origin_pos, player)

                elif not self.player_blue_turn and self.board[origin_pos[0]][origin_pos[1]] == 2:
                    self.draw_red(position_global)
                    self.execute_move(position_global, origin_pos, player)
        self.clear_possible_moves()
        self.window.bind("<Button-1>", self.click)


    def second_click_pressed(self, bool):
        if bool:
            return True
        return False


ataxx()

Escolha número de linhas/colunas:
3
Blue score=  5
Red score=  0
------------------------

BLUE WINS!!
Blue score=  9
Red score=  0
------------------------

BLUE WINS!!


### Server

In [1]:
import socket
import time
import numpy as np
from tkinter import *
import numpy as np
import copy
import random as r
import pickle


b_w="BLUE WINS!!"
r_w="RED WINS!!"
t = "IT'S A TIE!!"

print("Select board size (4 or 6): ")
NB = int(input())  # Board number of rows/columns
size_of_board = 600
size_of_square = size_of_board/NB
symbol_size = (size_of_square*0.75-10)/2
symbol_thickness = 20
blue_color = '#496BAB'
red_color = '#F33E30'

possible_moves_global=[]
position_global=[]
origin_pos=[]
moves_blue_global=[]
moves_red_global=[]
blue_pieces=[]
red_pieces=[]
board2=[]

class ServerAtaxx():
    def __init__(self):
        self.window = Tk()
        self.window.title('Ataxx')
        self.canvas = Canvas(self.window, width=size_of_board, height=size_of_board, background="white")
        self.canvas.pack()
        self.window.bind('<Button-1>', self.click)
        self.board = np.zeros(shape=(NB, NB))
        self.board[0][0]=2
        self.board[0][NB-1]=1
        self.board[NB-1][NB-1]=1
        self.board[NB-1][0]=2
        self.player_blue_turn = True
        self.game_ended = False
        self.human_has_played = False
        self.init_draw_board()

    #----------------DESENHO DO TABULEIRO---------------------------------------------------------------------------------------------------------

    def init_draw_board(self):
        self.canvas.delete("all")
        for i in range(NB-1):
            self.canvas.create_line((i+1)*size_of_square, 0, (i+1)*size_of_square, size_of_board)
        for i in range(NB-1):
            self.canvas.create_line(0,(i+1)*size_of_square, size_of_board, (i+1)*size_of_square)
        self.canvas.create_oval(size_of_square/2 - symbol_size, size_of_square/2 - symbol_size,
                                size_of_square/2 + symbol_size, size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=red_color,
                                fill=red_color)
        self.canvas.create_oval(size_of_board - size_of_square/2 - symbol_size,size_of_board - size_of_square/2 - symbol_size,
                                size_of_board - size_of_square/2 + symbol_size, size_of_board - size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=blue_color,
                                fill=blue_color)
        self.canvas.create_oval(size_of_square/2 - symbol_size,size_of_board - size_of_square/2 - symbol_size,
                                size_of_square/2 + symbol_size, size_of_board - size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=blue_color,
                                fill=blue_color)
        self.canvas.create_oval(size_of_board - size_of_square/2 - symbol_size, size_of_square/2- symbol_size,
                                size_of_board - size_of_square/2 + symbol_size, size_of_square/2 + symbol_size,
                                width=symbol_thickness, outline=red_color,
                                fill=red_color)


    def update_board(self, x, y, origin):
        if self.game_ended: return
        for i in range(max(0, x-1), min(NB, x+2)):
            for j in range(max(0, y-1), min(NB, y+2)):
                if not self.is_square_clear([i,j]):
                    if self.player_blue_turn:
                        self.draw_blue([i,j])
                    else:
                        self.draw_red([i,j])
                    self.board[i][j]=self.board[x][y]
        if x-origin[0]== 2 or y-origin[1]== 2 or x-origin[0]== -2 or y-origin[1]== -2:
            self.board[origin[0]][origin[1]]=0
            pos=self.convert_logical_to_grid_position(origin)
            self.draw_whitespace(pos)
        self.score()
        self.all_moves()
        self.player_blue_turn = not self.player_blue_turn

    def all_moves(self):
        global moves_blue_global
        global moves_red_global
        global blue_pieces
        global red_pieces
        for i in range(NB):
            for j in range(NB):
                if self.board[i][j]==1:
                    moves_blue_global.append(self.possible_moves([i,j]))
                    blue_pieces.append([i,j])
                elif self.board[i][j]==2:
                    moves_red_global.append(self.possible_moves([i,j]))
                    red_pieces.append([i,j])
        if len(moves_blue_global)==0:
            self.no_moves(1)
        elif len(moves_red_global)==0:
            self.no_moves(2)
        moves_blue_global=[]
        moves_red_global=[]

    def no_moves(self, player):
        if self.game_ended: return
        if player==1:
            for i in range(NB):
                for j in range(NB):
                    if self.board[i][j]==0:
                        self.board[i][j]=2
                        self.draw_red([i,j])
        elif player==2:
            for i in range(NB):
                for j in range(NB):
                    if self.board[i][j]==0:
                        self.board[i][j]=1
                        self.draw_blue([i,j])
        self.score()

    def execute_move(self, move, origin, player):
        self.board[move[0]][move[1]] = player
        self.update_board(move[0], move[1], origin)

    def is_square_clear(self, pos):
        if not np.array_equal(pos, []):
            return self.board[pos[0]][pos[1]] == 0

    def valid_move(self, logical_pos):
        return self.is_square_clear(logical_pos)

    #dado a peça selecionada, devolve uma lista
    #com todos os movimentos possiveis da mesma
    def possible_moves(self, move):
        possible_moves=[]
        for i in range(max(0,move[0]-2), min(NB, move[0]+3)):
            for j in range(max(0,move[1]-2), min(NB, move[1]+3)):

                if self.is_square_clear([i,j]):
                    possible_moves.append([i,j])

        return possible_moves

    def get_positions(self, player):
        pos = np.argwhere(self.board == player)
        return pos.tolist()

    def get_all_possible_moves(self, positions):
        all_moves = {}

        for pos in positions:
            moves = self.possible_moves(pos)
            all_moves[tuple(pos)] = moves

        return all_moves


    def score(self):
        cont_blue=0
        cont_red=0
        cheio=True
        for i in range(NB):
            for j in range(NB):
                if self.board[i][j]==1:
                    cont_blue+=1
                elif self.board[i][j]==2:
                    cont_red+=1
                if self.board[i][j]==0:
                    cheio=False
        print("Blue score= ",  cont_blue)
        print("Red score= ",  cont_red)
        print("------------------------")
        self.window.title("Ataxx - Red : %d vs %d : Blue" % (cont_red, cont_blue))
        if cont_blue==0:
            self.game_is_over(cont_red, cont_blue)
        elif cont_red==0:
            self.game_is_over(cont_red, cont_blue)
        elif cheio:
            self.game_is_over(cont_red, cont_blue)


    def game_is_over(self, red, blue):
        print()
        if blue>red:
            print(b_w)
        elif blue<red:
            print(r_w)
        else:
            print(t)
        self.game_ended = True

#----------------------TRANSFORMAR EM MATRIZ PARA APLICAR REGRAS---------------

    def convert_logical_to_grid_position(self, logical_pos):
        logical_pos = np.array(logical_pos, dtype=int)
        return np.array((size_of_square)*logical_pos + size_of_square/2)

    def convert_grid_to_logical_position(self, grid_pos):
        grid_pos = np.array(grid_pos)
        return np.array(grid_pos//size_of_square, dtype=int)

#-----------------------DESENHAR PECAS----------------------------------------
    def draw_whitespace(self, grid_pos):
        self.canvas.create_rectangle(grid_pos[0] - symbol_size, grid_pos[1] - symbol_size,
                            grid_pos[0] + symbol_size, grid_pos[1] + symbol_size,
                            width=symbol_thickness, outline="white",
                            fill="white")


    def draw_blue(self, logical_pos):
        logical_pos = np.array(logical_pos)
        grid_pos = self.convert_logical_to_grid_position(logical_pos)
        self.canvas.create_oval(grid_pos[0] - symbol_size, grid_pos[1] - symbol_size,
                            grid_pos[0] + symbol_size, grid_pos[1] + symbol_size,
                            width=symbol_thickness, outline=blue_color,
                            fill=blue_color)


    def draw_red(self, logical_pos):
        logical_pos = np.array(logical_pos)
        grid_pos = self.convert_logical_to_grid_position(logical_pos)
        self.canvas.create_oval(grid_pos[0] - symbol_size, grid_pos[1] - symbol_size,
                            grid_pos[0] + symbol_size, grid_pos[1] + symbol_size,
                            width=symbol_thickness, outline=red_color,
                            fill=red_color)


#----------------------------------- Verificaçao movimentos e jogadas ------------------------------------

    # desenha no tabuleiro as jogadas possiveis da bola selecionada
    def draw_possible_moves(self, possible_moves):
        moves=[0]*len(possible_moves)
        for i in range(len(possible_moves)):
            moves[i]=self.convert_logical_to_grid_position(possible_moves[i])
            self.canvas.create_oval(moves[i][0]-symbol_size, moves[i][1] - symbol_size,
                                    moves[i][0]+symbol_size, moves[i][1]+ symbol_size,
                                    width=symbol_thickness, outline="gray", fill="gray", tags="possible")
        self.window.update()


    def clear_possible_moves(self):
        self.canvas.delete("possible")

#----------------------- MOUSE -----------------------------------------------------------

    def handle_client(self, origin, logical_pos):
        if self.game_ended: return
        if self.board[origin[0]][origin[1]] == 1 and self.player_blue_turn:
            possible_moves_global = self.possible_moves(origin)
            time.sleep(1)
            self.draw_possible_moves(possible_moves_global)
            if not np.array_equal(possible_moves_global, []):
                self.handle_client2(origin, logical_pos)

        elif self.board[origin[0]][origin[1]] == 2 and not self.player_blue_turn:
            possible_moves_global = self.possible_moves(origin)
            time.sleep(1)
            self.draw_possible_moves(possible_moves_global)
            if not np.array_equal(possible_moves_global, []):
                self.handle_client2(origin, logical_pos)


    def handle_client2(self, origin_pos, position_global):
        if self.game_ended: return
        if self.player_blue_turn:
            player=1
        else:
            player=2
        if self.valid_move(position_global):
            if self.player_blue_turn and self.board[origin_pos[0]][origin_pos[1]] == 1:
                self.draw_blue(position_global)
                self.execute_move(position_global, origin_pos, player)

            elif not self.player_blue_turn and self.board[origin_pos[0]][origin_pos[1]] == 2:
                self.draw_red(position_global)
                self.execute_move(position_global, origin_pos, player)
        self.clear_possible_moves()

    def click(self, event):
        if self.game_ended: return
        grid_pos = [event.x, event.y]
        logical_pos = self.convert_grid_to_logical_position(grid_pos)
        global origin_pos
        global possible_moves_global
        origin_pos = logical_pos
        if self.board[logical_pos[0]][logical_pos[1]] == 1 and self.player_blue_turn:
            print(f"Origin: {origin_pos}")
            possible_moves_global = self.possible_moves(logical_pos)

            if not np.array_equal(possible_moves_global, []):
                self.window.bind("<Button-1>", self.second_click)
            self.draw_possible_moves(possible_moves_global)

        elif self.board[logical_pos[0]][logical_pos[1]] == 2 and not self.player_blue_turn:
            print(f"Origin: {origin_pos}")
            possible_moves_global = self.possible_moves(logical_pos)

            if not np.array_equal(possible_moves_global, []):
                self.window.bind("<Button-1>", self.second_click)
            self.draw_possible_moves(possible_moves_global)


    def second_click(self, event):
        if self.game_ended: return
        global bool
        grid_pos = [event.x, event.y]
        logical_pos = self.convert_grid_to_logical_position(grid_pos)
        global possible_moves_global
        possible_moves_global=np.array(possible_moves_global, dtype=int)

        for element in possible_moves_global:

            if np.array_equal(logical_pos, element):
                global position_global
                position_global = logical_pos
                bool=True
        bool = True
        print(f'Move: {position_global}')
        self.click2()
        possible_moves_global=[]
        position_global=[]
        self.human_has_played = True


    def click2(self):
        if self.game_ended: return
        global bool

        if self.player_blue_turn:
            player=1
        else:
            player=2

        if self.valid_move(position_global):

            if self.second_click_pressed(bool):
                if self.player_blue_turn and self.board[origin_pos[0]][origin_pos[1]] == 1:
                    self.draw_blue(position_global)
                    self.execute_move(position_global, origin_pos, player)

                elif not self.player_blue_turn and self.board[origin_pos[0]][origin_pos[1]] == 2:
                    self.draw_red(position_global)
                    self.execute_move(position_global, origin_pos, player)
        self.clear_possible_moves()
        self.window.bind("<Button-1>", self.click)


    def second_click_pressed(self, bool):
        if bool:
            return True
        return False


Game = "Ataxx"  # type of game

def start_server(host='localhost', port=5000):

    print("Select Player 1 (H for human, A for AI): ")
    player1_type = input().upper()
    print("Select Player 2 (H for human, A for AI): ")
    player2_type = input().upper()

    if player1_type=="H" and player2_type=="H":
        agent1 = 0
        agent2 = 0
    else:
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((host, port))
        server_socket.listen(2)

        if player1_type=="A" and player2_type=="A":
            print("Waiting for two agents to connect...")
            agent1, addr1 = server_socket.accept()
            print("Agent 1 connected from", addr1)
            bs = b'AG1 ' + Game.encode()
            agent1.sendall(bs)
            agent2, addr2 = server_socket.accept()
            print("Agent 2 connected from", addr2)
            bs = b'AG2 ' + Game.encode()
            agent2.sendall(bs)
        if player1_type=="A" and player2_type=="H":
            print("Waiting for one agent to connect...")
            agent1, addr1 = server_socket.accept()
            print("Agent 1 connected from", addr1)
            bs = b'AG1 ' + Game.encode()
            agent1.sendall(bs)
            agent2 = 0
        if player1_type=="H" and player2_type=="A":
            print("Waiting for one agent to connect...")
            agent1 = 0
            agent2, addr2 = server_socket.accept()
            print("Agent 2 connected from", addr2)
            bs = b'AG2 ' + Game.encode()
            agent2.sendall(bs)

    print("------------------------")
    agents = [agent1, agent2]
    current_agent = 0

    jog = 0

    game = ServerAtaxx()

    while not game.game_ended:
        game.window.update()
        if agents[current_agent]!=0:
            posicoes = game.get_positions(current_agent+1)
            all_pos = game.get_all_possible_moves(posicoes)
            all_pos_bytes = pickle.dumps(all_pos)
            agents[current_agent].sendall(all_pos_bytes)
            try:
                data = agents[current_agent].recv(1024)
                selected_key, selected_object = pickle.loads(data)
                origin = np.array(selected_key)
                move = np.array(selected_object)
                if not data:
                    break

                # Process the move
                print("Agent", current_agent+1, ": ")
                print(f"Origin: {origin}")
                print(f'Move: {move}')
                game.handle_client(origin, move)
                if agents[1-current_agent]==0:
                    print("Agent", 1-current_agent+1, ": ")


                # Switch to the other agent
                current_agent = 1 - current_agent
                jog += 1
                if game.game_ended:
                    break
                time.sleep(1)

            except Exception as e:
                print("Error:", e)
                break
        if game.human_has_played:
            current_agent = 1 - current_agent
            game.human_has_played = False


    print("\n-----------------\nGAME END\n-----------------\n")
    time.sleep(1)
    if agent1!=0:
        agent1.close()
    if agent2!=0:
        agent2.close()
    server_socket.close()

if __name__ == "__main__":
    start_server()

Select board size (4 or 6): 
4
Select Player 1 (H for human, A for AI): 
H
Select Player 2 (H for human, A for AI): 
A
Waiting for one agent to connect...
Agent 2 connected from ('127.0.0.1', 17865)
------------------------
Origin: [3 3]
Move: [2 2]
Blue score=  3
Red score=  2
------------------------
Agent 2 : 
Origin: [0 0]
Move: [0 2]
Blue score=  2
Red score=  3
------------------------
Agent 1 : 
Origin: [2 2]
Move: [1 2]
Blue score=  5
Red score=  1
------------------------
Agent 2 : 
Origin: [3 0]
Move: [2 1]
Blue score=  3
Red score=  4
------------------------
Agent 1 : 
Origin: [3 3]
Move: [3 2]
Blue score=  6
Red score=  2
------------------------
Agent 2 : 
Origin: [1 2]
Move: [0 0]
Blue score=  6
Red score=  2
------------------------
Agent 1 : 
Origin: [2 2]
Move: [1 2]
Blue score=  7
Red score=  2
------------------------
Agent 2 : 
Origin: [0 0]
Move: [2 0]
Blue score=  6
Red score=  3
------------------------
Agent 1 : 
Origin: [1 2]
Move: [1 1]
Blue score=  9
Red sco

## Jogo Go

Interface gráfica para o jogo Go.

In [ ]:
import pygame
import numpy as np
import itertools
import sys
import networkx as nx
import collections
from pygame import gfxdraw
import time

# Define board size
print("Select board size (7 or 9): ")
size = int (input())

# Game constantes
BOARD_BROWN = (141, 104, 75)    # Change color as desired
BOARD_WIDTH = 800           #New size of board
BOARD_BORDER = 75
STONE_RADIUS = int (abs(BOARD_WIDTH/size*20*0.02))      #Ajust stone size to grid size
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
TURN_POS = (BOARD_BORDER, 20)
SCORE_POS = (BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER + 30)
DOT_RADIUS = 2


def make_grid(size):
    """Return list of (start_point, end_point pairs) defining gridlines

    Args:
        size (int): size of grid

    Returns:
        Tuple[List[Tuple[float, float]]]: start and end points for gridlines
    """
    start_points, end_points = [], []

    # vertical start points (constant y)
    xs = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    ys = np.full((size), BOARD_BORDER)
    start_points += list(zip(xs, ys))

    # horizontal start points (constant x)
    xs = np.full((size), BOARD_BORDER)
    ys = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    start_points += list(zip(xs, ys))

    # vertical end points (constant y)
    xs = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    ys = np.full((size), BOARD_WIDTH - BOARD_BORDER)
    end_points += list(zip(xs, ys))

    # horizontal end points (constant x)
    xs = np.full((size), BOARD_WIDTH - BOARD_BORDER)
    ys = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    end_points += list(zip(xs, ys))

    return (start_points, end_points)


class HumanPlayer:

    def make_move(self, board):
        # Check if the current player is an AI player
        if isinstance(self, AI):
            # Place a white stone on the board
            board.place_stone((0, 0), WHITE)
        else:
            # For a human player, you might want to implement a way to get the
            # move from the user
            pass

class AI:
    def __init__(self, color):
        self.color = color

    def make_move(self, board):
        # Implement AI move generation logic here
        # Construct alphaZero on the side and import here

        # random move generator (valid moves only)
        valid_moves = [(col, row) for col in range(board.shape[0]) for row in range(board.shape[1]) if board[col, row] == 0]
        if not valid_moves:
            return None  # No valid moves available

        return valid_moves[np.random.choice(len(valid_moves))]


def xy_to_colrow(x, y, size):
    """Convert x,y coordinates to column and row number

    Args:
        x (float): x position
        y (float): y position
        size (int): size of grid

    Returns:
        Tuple[int, int]: column and row numbers of intersection
    """
    inc = (BOARD_WIDTH - 2 * BOARD_BORDER) / (size - 1)
    x_dist = x - BOARD_BORDER
    y_dist = y - BOARD_BORDER
    col = int(round(x_dist / inc))
    row = int(round(y_dist / inc))
    return col, row


def colrow_to_xy(col, row, size):
    """Convert column and row numbers to x,y coordinates

    Args:
        col (int): column number (horizontal position)
        row (int): row number (vertical position)
        size (int): size of grid

    Returns:
        Tuple[float, float]: x,y coordinates of intersection
    """
    inc = (BOARD_WIDTH - 2 * BOARD_BORDER) / (size - 1)
    x = int(BOARD_BORDER + col * inc)
    y = int(BOARD_BORDER + row * inc)
    return x, y


def has_no_liberties(board, group):
    """Check if a stone group has any liberties on a given board.

    Args:
        board (object): game board (size * size matrix)
        group (List[Tuple[int, int]]): list of (col,row) pairs defining a stone group

    Returns:
        [boolean]: True if group has any liberties, False otherwise
    """
    for x, y in group:
        if x > 0 and board[x - 1, y] == 0:
            return False
        if y > 0 and board[x, y - 1] == 0:
            return False
        if x < board.shape[0] - 1 and board[x + 1, y] == 0:
            return False
        if y < board.shape[0] - 1 and board[x, y + 1] == 0:
            return False
    return True


def get_stone_groups(board, color):
    """Get stone groups of a given color on a given board

    Args:
        board (object): game board (size * size matrix)
        color (str): name of color to get groups for

    Returns:
        List[List[Tuple[int, int]]]: list of list of (col, row) pairs, each defining a group
    """
    size = board.shape[0]
    color_code = 1 if color == "black" else 2
    xs, ys = np.where(board == color_code)
    graph = nx.grid_graph(dim=[size, size])
    stones = set(zip(xs, ys))
    all_spaces = set(itertools.product(range(size), range(size)))
    stones_to_remove = all_spaces - stones
    graph.remove_nodes_from(stones_to_remove)
    return nx.connected_components(graph)


def is_valid_move(col, row, board):
    """Check if placing a stone at (col, row) is valid on board

    Args:
        col (int): column number
        row (int): row number
        board (object): board grid (size * size matrix)

    Returns:
        boolean: True if move is valid, False otherewise
    """
    # TODO: check for ko situation (infinite back and forth)
    if col < 0 or col >= board.shape[0]:
        return False
    if row < 0 or row >= board.shape[0]:
        return False
    return board[col, row] == 0


class Game:
    def __init__(self, size, player1, player2):
        self.board = np.zeros((size, size))
        self.size = size
        self.black_turn = True
        self.prisoners = collections.defaultdict(int)
        self.start_points, self.end_points = make_grid(self.size)
        self.player1 = player1
        self.player2 = player2
        self.player1_passed = False
        self.player2_passed = False
        self.current_player = self.player1 if self.black_turn else self.player2

    def init_pygame(self):
        pygame.init()
        screen = pygame.display.set_mode((BOARD_WIDTH, BOARD_WIDTH))
        self.screen = screen
        self.ZOINK = pygame.mixer.Sound("wav/zoink.wav")
        self.CLICK = pygame.mixer.Sound("wav/click.wav")
        self.font = pygame.font.SysFont("arial", 30)


    def clear_screen(self):

        # fill board and add gridlines
        self.screen.fill(BOARD_BROWN)
        for start_point, end_point in zip(self.start_points, self.end_points):
            pygame.draw.line(self.screen, BLACK, start_point, end_point)

        # add guide dots
        guide_dots = [3, self.size // 2, self.size - 4]
        for col, row in itertools.product(guide_dots, guide_dots):
            x, y = colrow_to_xy(col, row, self.size)
            gfxdraw.aacircle(self.screen, x, y, DOT_RADIUS, BLACK)
            gfxdraw.filled_circle(self.screen, x, y, DOT_RADIUS, BLACK)

        pygame.display.flip()

    def check_end_game(self):
        # Check if both players passed or there are no more valid moves
        if self.passed_twice() or not any(is_valid_move(col, row, self.board) for col in range(self.size) for row in range(self.size)):
            return True
        return False

    def passed_twice(self):
        # Check if both players passed their turn twice in a row
        return self.player1_passed and self.player2_passed


    def pass_move(self):
        if self.black_turn:
            self.player1_passed = True
        else:
            self.player2_passed = True

        self.black_turn = not self.black_turn
        self.draw()

        # Check for end-of-game condition after a pass
        if self.check_end_game():
            self.print_final_scores()
            pygame.quit()
            sys.exit()

    def print_final_scores(self):
        score_black = self.prisoners['black']
        score_white = self.prisoners['white']
        print("Game Over!")
        print(f"Black's Prisoners: {score_black}")
        print(f"White's Prisoners: {score_white}")
        if score_black > score_white:
            print("Black wins!")
        elif score_black < score_white:
            print("White wins!")
        else:
            print("It's a tie!")

    def update(self):
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.MOUSEBUTTONUP:
                self.handle_click()
            if event.type == pygame.QUIT:
                self.print_final_scores()
                sys.exit()
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_p:
                    self.pass_move()

    def handle_click(self):
        # get board position
        x, y = pygame.mouse.get_pos()
        col, row = xy_to_colrow(x, y, self.size)
        if not is_valid_move(col, row, self.board):
            self.ZOINK.play()
            return

        # update board array
        #self.board[col, row] = 1 if self.black_turn else 2
        self.board[col, row] = 1 if self.current_player == self.player1 else 2

        self.handle_captures(col, row, "black" if self.current_player == self.player1 else "white")

        # change turns and draw screen
        self.CLICK.play()
        self.black_turn = not self.black_turn
        self.current_player = self.player1 if self.black_turn else self.player2  # Update current player
        self.draw()

    def handle_captures(self, col, row, color):
        # Handle captures for a given move at (col, row) and color
        other_color = "white" if color == "black" else "black"
        capture_happened = False

        for group in list(get_stone_groups(self.board, other_color)):
            if has_no_liberties(self.board, group):
                capture_happened = True
                for i, j in group:
                    self.board[i, j] = 0
                self.prisoners[color] += len(group)

        if not capture_happened:
            group = None
            for group in get_stone_groups(self.board, color):
                if (col, row) in group:
                    break
            if has_no_liberties(self.board, group):
                if not self.current_player_is_human():
                    player_name = "Player 1" if color == "black" else "Player 2"
                    print(f"{player_name} has no liberties.")
                self.ZOINK.play()
                self.board[col, row] = 0
                return


    def draw(self):
        # draw stones - filled circle and antialiased ring
        self.clear_screen()
        for col, row in zip(*np.where(self.board == 1)):
            x, y = colrow_to_xy(col, row, self.size)
            gfxdraw.aacircle(self.screen, x, y, STONE_RADIUS, BLACK)
            gfxdraw.filled_circle(self.screen, x, y, STONE_RADIUS, BLACK)
        for col, row in zip(*np.where(self.board == 2)):
            x, y = colrow_to_xy(col, row, self.size)
            gfxdraw.aacircle(self.screen, x, y, STONE_RADIUS, WHITE)
            gfxdraw.filled_circle(self.screen, x, y, STONE_RADIUS, WHITE)

        # text for score and turn info
        score_msg = (
            f"Black's Prisoners: {self.prisoners['black']}"
            + f"     White's Prisoners: {self.prisoners['white']}"
        )
        txt = self.font.render(score_msg, True, BLACK)
        self.screen.blit(txt, SCORE_POS)
        turn_msg = (
            f"{'Black' if self.black_turn else 'White'} to move. "
            + "Click to place stone, press P to pass."
        )
        txt = self.font.render(turn_msg, True, BLACK)
        self.screen.blit(txt, TURN_POS)

        pygame.display.flip()

    def current_player_is_human(self):
        return self.current_player is None or isinstance(self.current_player, HumanPlayer)


if __name__ == "__main__":
    print("Select Player 1 (H for human, A for AI): ")
    player1_type = input().upper()
    player1 = HumanPlayer() if player1_type == "H" else AI(color="black")

    print("Select Player 2 (H for human, A for AI): ")
    player2_type = input().upper()
    player2 = HumanPlayer() if player2_type == "H" else AI(color="white")

    g = Game(size, player1, player2)

    g.init_pygame()
    g.clear_screen()
    g.draw()

    while True:

        if g.current_player_is_human():
            g.update()

        # Check for game end conditions here if needed
        if not g.current_player_is_human():
            time.sleep(1)
            move = g.current_player.make_move(g.board)
            # Check if the move is valid
            if move is not None and is_valid_move(*move, g.board):
                g.board[move[0], move[1]] = 1 if g.current_player == g.player1 else 2
                g.black_turn = not g.black_turn  # Explicitly update the turn after the human move
                g.current_player = g.player1 if g.black_turn else g.player2
                g.handle_captures(move[0], move[1], "black" if g.current_player == g.player2 else "white")
                g.CLICK.play()
                g.draw()

            else:
                g.pass_move()
                print("AI couldn't find a valid move.")


        pygame.time.wait(100)

### Server

In [ ]:
import pygame
import numpy as np
import itertools
import sys
import networkx as nx
import collections
from pygame import gfxdraw
import time
import socket
import time
import copy
import random
import pickle

# Define board size
print("Select board size (7 or 9): ")
size = int (input())

# Game constantes
BOARD_BROWN = (141, 104, 75)    # Change color as desired
BOARD_WIDTH = 600           #New size of board
BOARD_BORDER = 75
STONE_RADIUS = int (abs(BOARD_WIDTH/size*20*0.02))      #Ajust stone size to grid size
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
TURN_POS = (BOARD_BORDER, 20)
SCORE_POS = (BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER + 30)
DOT_RADIUS = 2


def make_grid(size):
    """Return list of (start_point, end_point pairs) defining gridlines

    Args:
        size (int): size of grid

    Returns:
        Tuple[List[Tuple[float, float]]]: start and end points for gridlines
    """
    start_points, end_points = [], []

    # vertical start points (constant y)
    xs = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    ys = np.full((size), BOARD_BORDER)
    start_points += list(zip(xs, ys))

    # horizontal start points (constant x)
    xs = np.full((size), BOARD_BORDER)
    ys = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    start_points += list(zip(xs, ys))

    # vertical end points (constant y)
    xs = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    ys = np.full((size), BOARD_WIDTH - BOARD_BORDER)
    end_points += list(zip(xs, ys))

    # horizontal end points (constant x)
    xs = np.full((size), BOARD_WIDTH - BOARD_BORDER)
    ys = np.linspace(BOARD_BORDER, BOARD_WIDTH - BOARD_BORDER, size)
    end_points += list(zip(xs, ys))

    return (start_points, end_points)


def xy_to_colrow(x, y, size):
    """Convert x,y coordinates to column and row number

    Args:
        x (float): x position
        y (float): y position
        size (int): size of grid

    Returns:
        Tuple[int, int]: column and row numbers of intersection
    """
    inc = (BOARD_WIDTH - 2 * BOARD_BORDER) / (size - 1)
    x_dist = x - BOARD_BORDER
    y_dist = y - BOARD_BORDER
    col = int(round(x_dist / inc))
    row = int(round(y_dist / inc))
    return col, row


def colrow_to_xy(col, row, size):
    """Convert column and row numbers to x,y coordinates

    Args:
        col (int): column number (horizontal position)
        row (int): row number (vertical position)
        size (int): size of grid

    Returns:
        Tuple[float, float]: x,y coordinates of intersection
    """
    inc = (BOARD_WIDTH - 2 * BOARD_BORDER) / (size - 1)
    x = int(BOARD_BORDER + col * inc)
    y = int(BOARD_BORDER + row * inc)
    return x, y


def has_no_liberties(board, group):
    """Check if a stone group has any liberties on a given board.

    Args:
        board (object): game board (size * size matrix)
        group (List[Tuple[int, int]]): list of (col,row) pairs defining a stone group

    Returns:
        [boolean]: True if group has any liberties, False otherwise
    """
    for x, y in group:
        if x > 0 and board[x - 1, y] == 0:
            return False
        if y > 0 and board[x, y - 1] == 0:
            return False
        if x < board.shape[0] - 1 and board[x + 1, y] == 0:
            return False
        if y < board.shape[0] - 1 and board[x, y + 1] == 0:
            return False
    return True


def get_stone_groups(board, color):
    """Get stone groups of a given color on a given board

    Args:
        board (object): game board (size * size matrix)
        color (str): name of color to get groups for

    Returns:
        List[List[Tuple[int, int]]]: list of list of (col, row) pairs, each defining a group
    """
    size = board.shape[0]
    color_code = 1 if color == "black" else 2
    xs, ys = np.where(board == color_code)
    graph = nx.grid_graph(dim=[size, size])
    stones = set(zip(xs, ys))
    all_spaces = set(itertools.product(range(size), range(size)))
    stones_to_remove = all_spaces - stones
    graph.remove_nodes_from(stones_to_remove)
    return nx.connected_components(graph)


def is_valid_move(col, row, board):
    """Check if placing a stone at (col, row) is valid on board

    Args:
        col (int): column number
        row (int): row number
        board (object): board grid (size * size matrix)

    Returns:
        boolean: True if move is valid, False otherewise
    """
    # TODO: check for ko situation (infinite back and forth)
    if col < 0 or col >= board.shape[0]:
        return False
    if row < 0 or row >= board.shape[0]:
        return False
    return board[col, row] == 0


class ServerGo:
    def __init__(self, size, player1, player2):
        self.board = np.zeros((size, size))
        self.size = size
        self.black_turn = True
        self.prisoners = collections.defaultdict(int)
        self.start_points, self.end_points = make_grid(self.size)
        self.player1 = player1
        self.player2 = player2
        self.player1_passed = False
        self.player2_passed = False
        self.current_player = self.player1 if self.black_turn else self.player2
        self.human_has_played = False

    def init_pygame(self):
        pygame.init()
        screen = pygame.display.set_mode((BOARD_WIDTH, BOARD_WIDTH))
        self.screen = screen
        self.ZOINK = pygame.mixer.Sound("wav/zoink.wav")
        self.CLICK = pygame.mixer.Sound("wav/click.wav")
        self.font = pygame.font.SysFont("arial", 30)


    def clear_screen(self):

        # fill board and add gridlines
        self.screen.fill(BOARD_BROWN)
        for start_point, end_point in zip(self.start_points, self.end_points):
            pygame.draw.line(self.screen, BLACK, start_point, end_point)

        # add guide dots
        guide_dots = [3, self.size // 2, self.size - 4]
        for col, row in itertools.product(guide_dots, guide_dots):
            x, y = colrow_to_xy(col, row, self.size)
            gfxdraw.aacircle(self.screen, x, y, DOT_RADIUS, BLACK)
            gfxdraw.filled_circle(self.screen, x, y, DOT_RADIUS, BLACK)

        pygame.display.flip()


    def check_end_game(self):
        # Check if both players passed or there are no more valid moves
        if self.passed_twice() or not any(is_valid_move(col, row, self.board) for col in range(self.size) for row in range(self.size)):
            print('aqui')
            return True
        return False

    def passed_twice(self):
        # Check if both players passed their turn twice in a row
        return self.player1_passed and self.player2_passed


    def pass_move(self):
        if self.black_turn:
            self.player1_passed = True
        else:
            self.player2_passed = True

        self.black_turn = not self.black_turn
        self.draw()

        # Check for end-of-game condition after a pass
        if self.check_end_game():
            self.print_final_scores()
            pygame.quit()
            sys.exit()

    def print_final_scores(self):
        score_black = self.prisoners['black']
        score_white = self.prisoners['white']
        print("Game Over!")
        print(f"Black's Prisoners: {score_black}")
        print(f"White's Prisoners: {score_white}")
        if score_black > score_white:
            print("Black wins!")
        elif score_black < score_white:
            print("White wins!")
        else:
            print("It's a tie!")

    def print_scores(self):
        score_black = self.prisoners['black']
        score_white = self.prisoners['white']
        print(f"Black's Prisoners: {score_black}")
        print(f"White's Prisoners: {score_white}")
        print("------------------------")

    def update(self):
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.MOUSEBUTTONUP:
                self.handle_click()
            if event.type == pygame.QUIT:
                self.print_final_scores()
                sys.exit()
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_p:
                    self.pass_move()

    def handle_click(self):
        # get board position
        x, y = pygame.mouse.get_pos()
        col, row = xy_to_colrow(x, y, self.size)
        if not is_valid_move(col, row, self.board):
            self.ZOINK.play()
            return

        # update board array
        #self.board[col, row] = 1 if self.black_turn else 2
        self.board[col, row] = 1 if self.current_player == self.player1 else 2

        self.handle_captures(col, row, "black" if self.current_player == self.player1 else "white")
        # change turns and draw screen
        self.CLICK.play()
        self.black_turn = not self.black_turn
        self.current_player = self.player1 if self.black_turn else self.player2  # Update current player
        self.draw()
        print(f'Move: ({col}, {row})')
        self.print_scores()
        self.human_has_played = True


    def handle_captures(self, col, row, color):
        # Handle captures for a given move at (col, row) and color
        other_color = "white" if color == "black" else "black"
        capture_happened = False

        for group in list(get_stone_groups(self.board, other_color)):
            if has_no_liberties(self.board, group):
                capture_happened = True
                for i, j in group:
                    self.board[i, j] = 0
                self.prisoners[color] += len(group)

        if not capture_happened:
            group = None
            for group in get_stone_groups(self.board, color):
                if (col, row) in group:
                    break
            if has_no_liberties(self.board, group):
                if not self.current_player_is_human():
                    player_name = "Player 1" if color == "black" else "Player 2"
                    print(f"{player_name} has no liberties.")
                self.ZOINK.play()
                self.board[col, row] = 0
                return


    def draw(self):
        # draw stones - filled circle and antialiased ring
        self.clear_screen()
        for col, row in zip(*np.where(self.board == 1)):
            x, y = colrow_to_xy(col, row, self.size)
            gfxdraw.aacircle(self.screen, x, y, STONE_RADIUS, BLACK)
            gfxdraw.filled_circle(self.screen, x, y, STONE_RADIUS, BLACK)
        for col, row in zip(*np.where(self.board == 2)):
            x, y = colrow_to_xy(col, row, self.size)
            gfxdraw.aacircle(self.screen, x, y, STONE_RADIUS, WHITE)
            gfxdraw.filled_circle(self.screen, x, y, STONE_RADIUS, WHITE)

        # text for score and turn info
        score_msg = (
            f"Black's Prisoners: {self.prisoners['black']}"
            + f"     White's Prisoners: {self.prisoners['white']}"
        )
        txt = self.font.render(score_msg, True, BLACK)
        self.screen.blit(txt, SCORE_POS)
        turn_msg = (
            f"{'Black' if self.black_turn else 'White'} to move. "
            + "Click to place stone, press P to pass."
        )
        txt = self.font.render(turn_msg, True, BLACK)
        self.screen.blit(txt, TURN_POS)

        pygame.display.flip()

    def current_player_is_human(self):
        return self.current_player is None or self.current_player==0 or self.current_player==1

Game = "Go"  # type of game

def start_server(host='localhost', port=5000):

    print("Select Player 1 (H for human, A for AI): ")
    player1_type = input().upper()
    print("Select Player 2 (H for human, A for AI): ")
    player2_type = input().upper()

    if player1_type=="H" and player2_type=="H":
        agent1 = 0
        agent2 = 1
    else:
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((host, port))
        server_socket.listen(2)

        if player1_type=="A" and player2_type=="A":
            print("Waiting for two agents to connect...")
            agent1, addr1 = server_socket.accept()
            print("Agent 1 connected from", addr1)
            bs = b'AG1 ' + Game.encode()
            agent1.sendall(bs)
            agent2, addr2 = server_socket.accept()
            print("Agent 2 connected from", addr2)
            bs = b'AG2 ' + Game.encode()
            agent2.sendall(bs)
        if player1_type=="A" and player2_type=="H":
            print("Waiting for one agent to connect...")
            agent1, addr1 = server_socket.accept()
            print("Agent 1 connected from", addr1)
            bs = b'AG1 ' + Game.encode()
            agent1.sendall(bs)
            agent2 = 0
        if player1_type=="H" and player2_type=="A":
            print("Waiting for one agent to connect...")
            agent1 = 0
            agent2, addr2 = server_socket.accept()
            print("Agent 2 connected from", addr2)
            bs = b'AG2 ' + Game.encode()
            agent2.sendall(bs)
            print("Agent 1 : ")

    print("------------------------")
    agents = [agent1, agent2]
    current_agent = 0

    jog = 0
    game = ServerGo(size, agent1, agent2)
    game.init_pygame()
    game.clear_screen()
    game.draw()
    while not game.check_end_game():
        if agents[current_agent]==0 or agents[current_agent]==1:
            game.update()
        if agents[current_agent]!=0 and agents[current_agent]!=1:
            board_byter = pickle.dumps(game.board)
            agents[current_agent].sendall(board_byter)
            try:
                data = agents[current_agent].recv(1024)
                move = pickle.loads(data)
                if not move:
                    break
                time.sleep(1)
                print("Agent", current_agent+1, ": ")
                print(f'Move: {move}')
                # Check if the move is valid
                if move is not None and is_valid_move(*move, game.board):
                    game.board[move[0], move[1]] = 1 if game.current_player == game.player1 else 2
                    game.black_turn = not game.black_turn  # Explicitly update the turn after the human move
                    game.current_player = game.player1 if game.black_turn else game.player2
                    game.handle_captures(move[0], move[1], "black" if game.current_player == game.player2 else "white")
                    game.CLICK.play()
                    game.draw()
                    game.print_scores()

                else:
                    game.pass_move()
                    print("AI couldn't find a valid move.")

                if agents[1-current_agent]==0 or agents[current_agent]==1:
                    print("Agent", 1-current_agent+1, ": ")


                # Switch to the other agent
                current_agent = 1 - current_agent
                jog += 1

            except Exception as e:
                print("Error:", e)
                break
        if game.human_has_played:
            current_agent = 1 - current_agent
            game.human_has_played = False


    print("\n-----------------\nGAME END\n-----------------\n")
    time.sleep(1)
    agent1.close()
    agent2.close()
    server_socket.close()

if __name__ == "__main__":
    start_server()